Supppose we ask a complex question to many people and aggregate their answers. In most cases this will produce better results than one person's (even expert) answer. THis is the *wisdom of the crowd*.Similarly, aggregating predictions of a group of predictors will many times lead to the best overall predictions. This is called *Ensemble Learning*, and an EL algorithm is called an Ensemble method.

One example of this is training many decision trees based on different random subsets of the training set. To get the final prediciton you get the results of all the trees and output the one with most votes. This is a *Random Forest*, and is a very powerful ML algorithm.

We will often use EL methods near theend of a project when we already have a few good predictors, to combine them. Many winners of ML competitions use several ensemble methods. 

In this chapter we will discuss the most popular methods including bagging, boosting, and stacking.

## Voting classifiers

Say we have a few predictors with 80% accuracy: SVM, RF, Log, KNN, etc..

<img src="CH 07 Images/01.PNG" />

We can create a better predictor by getting the predictions of each of these and outputing the class with the most votes. This is called a *hard voting* classifier.

<img src="CH 07 Images/02.PNG" />

This classifier often achieves a higher accuracy than even the best of our individual classifiers. In fact even if each classifier is a weak learner (only does slightly better than random guessing), the ensemble can still be a strong learner (achieving high accuracy). preovided there are suffiient weak learners and they are diverse.

Let's make sense of how this can happen. Suppose we have a bias coin with 51% chance of coming up heads. 1000 tosses will yield 510 heads. The probability of getting a majority of heads after 1000 tosses is 75%, and this number rises as you toss more. This is due to the law of large numbers: the more we toss the coin the closer we will be to the 51% heads probability as shown below for 10 different series.

<img src="CH 07 Images/03.PNG" />

Similarly, we can build 1000 classifiers that are individually correct 51% of the time, but when combined can have a 75% accuracy. This s only true if the classifiers are perfectly independent, making uncorrelated errors, which isn't the case because they are trained on the same data. This leads to having similar errors, many votes for the wrong class, and less accuracy.

**Tip: EL works best when predictors are independent. One way to get diverse classifiers is to train them on very different algorithms.**

The following cretes a voting classifier:

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [45]:
log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC(probability=True)

In [54]:
##train data here
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
X,y = make_moons( n_samples = 1000, noise = 0.15) 
#polynomial_svm_clf = Pipeline([("poly_features", PolynomialFeatures( degree = 3)), ("scaler", StandardScaler()), ("svm_clf", LinearSVC( C = 10, loss ="hinge")) ])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [47]:
voting_clf = VotingClassifier(estimators=[('lr',log_clf),('rf',rnd_clf),('svc',svm_clf)],voting='hard')
voting_clf.fit(X_train, y_train)

C:\Users\Allan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Allan\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Allan\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


VotingClassifier(estimators=[('lr',
                              LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='warn',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='warn', tol=0.0001,
                                                 verbose=0, warm_start=False)),
                             ('rf',
                              RandomForestClassifier(bootstrap=True,
                                                     class_weight=None,
                                                     criterion='gini',...
                                        

In [48]:
from sklearn.metrics import accuracy_score
for clf in (log_clf,rnd_clf,svm_clf,voting_clf):
    clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    print( clf.__class__.__name__, accuracy_score( y_test, y_pred))



LogisticRegression 0.9
RandomForestClassifier 1.0
SVC 0.95
VotingClassifier 0.95


C:\Users\Allan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Allan\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Allan\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\Allan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Allan\Anaconda3\lib\site-packages\sklearn

(something with data import wasn't consistent with book, otherwise):

We see the coting classifier performs slightly better than the individual ones.

If all estimators canestimate lass probabilities, you can tell sklearn to predict the class with the highest class probability aceraged over all the individual classifiers. This is called *soft voting*. It often achieves better performance than hard voting because it gives more weight to highly confident votes. Just put soft instead of hard and ensure all classifiers can estimate class probabilities. SVC cannot by default, so set its probability hyperparm to True.

## Bagging and Pasting

One way to get diverse classifiers is using different classifiers; another approach is to use the same algorithm for every predictor and train them on different random subsets of training data. When this is done *with replacement* it is called **bagging** when it is done *without replacement* it is called **pasting**.

Thus only bagging allows training instances to be sampled several times for the same predictor as shown below.

<img src="CH 07 Images/04.PNG" />

Afer the models are trained, outputs are aggregated using a function called a *statistical mode* (most frequent prediction) for classification of the average for regression. Each predictor has higher bias than if it were trained on the whole training set, but the aggregation reduces bias and variance. Generally the end results has similar bias and lower various than a single predictor on the whole set.

As seen above, the predictors can be trained in parallel and thus on multiple CPU cores or servers. This is why the methods are so popular, as it means they can scale well.

### Bagging and pasting in SKlearn

There is an API for bagging and pasting with BaggingClassifier or BagingRegresspor. the following code trains 500 trees: each on 100 instances randomly sampled from the data [using bagging, for pasting set bootstrap=False]. The m_jobs tells sklearn the number of CPI cores to use [-1 means use all available cores].


In [50]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

In [52]:
bag_clf = BaggingClassifier(DecisionTreeClassifier(),n_estimators=500,max_samples=100,bootstrap=True,n_jobs=-1)

In [57]:
bag_clf.fit(X_train,y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight=None,
                                                        criterion='gini',
                                                        max_depth=None,
                                                        max_features=None,
                                                        max_leaf_nodes=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=1,
                                                        min_samples_split=2,
                                                        min_weight_fraction_leaf=0.0,
                                                        presort=False,
                                                        random_state=None,
                                                        splitter='best'),
    

In [60]:
y_pred=bag_clf.predict(X_test)
accuracy_score(y_pred,y_test)

0.97

Below we see the decision boundary of a single tree compares with the bagging method on the same dataset. The ensemble's predictions generalize better: has comarable bias but smaller variance (~same numer of errors, but the boundary is more regular).

<img src="CH 07 Images/05.PNG" />

Bootstrapping introduces more diversiry, so bagging has slightly better bias than pasting; htis also means the predictors are less correlated so the variance is reduced. Overall bagging is better, but if you have time and CPU you can do both pagging and pasting and select the best method.

### Outof Bag evaluation

In bagging, some instances may be sampled multiple times while others aren't sampled at all. A bagging classifier samples m training instances with replacement, where m is the size of the training set. On average 63% of samples are used for each predictor, and the remaining ones are out of bag instances [different for each predictor].

A predictor doesn't see OOB instances and can't evaluate on them. You can evaluate the ensemble by averaging the oob evaluations of each predictor.

In sklearn you can set oob_score=True to request evaluation after training, and the result is accessed with oob_score_:

In [63]:
bag_clf = BaggingClassifier(DecisionTreeClassifier(),n_estimators=500,max_samples=100,bootstrap=True,n_jobs=-1,oob_score=True)
bag_clf.fit(X_train,y_train)
bag_clf.oob_score_

0.97625

According to the above, the classifier is likely to achieve 97% accuracy on the test set. Let's verify:

In [66]:
from sklearn.metrics import accuracy_score
y_pred = bag_clf.predict(X_test)
accuracy_score(y_test,y_pred)
#97%. close enough

0.97

oob decision function for each training instance can also be accessed with oob_decision_function_. In this case it returns class probabilities for each training instance.

In [68]:
bag_clf.oob_decision_function_
#eg it classifies first instance as 99.77% belonging to negative class

array([[0.00225734, 0.99774266],
       [0.99111111, 0.00888889],
       [0.81715576, 0.18284424],
       ...,
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.93396226, 0.06603774]])

## Random Patches and Random Subspaces

BaggingClassifier class supports sampling the features as well. It is controlled by two hyperparams: max_features, bootstrap_features. They work like max_samples and bootstrap, but for feature sampling instead of instance sampling. Thus each predictor will be trained on a random subset of input features.

This is useful when dealing with high dimensional imput like images. Sampling both instances and features is aclled *Random patches* method. Keeping all the training instances [bootstrap=F, max samples=1], but sampling features is called the *random subspace* method. 

Sampling features results in more diversity, trading off more bias for lower variance.

### Random Forests

RF is a type of ensemble of trees generally trained with bagging and with max_samples=size of training set.

Use this to train many sets of trees instead of baggingclassifier.

In [70]:
from sklearn.ensemble import RandomForestClassifier
rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16,n_jobs=-1)
rnd_clf.fit(X_train,y_train)
y_pred_rf = rnd_clf.predict(X_test)

### Extra-Trees

During tree growth, at each node a random subset of features is considered for the splitting. You can also increase randomness by using random thresholds for each feature rather than searching for the best thresholds. A tree with this added randomness is called an *Extremely Randomized Tree* ensemble [Extra-Trees]. Yields more bias and lower variance. Also makes the method faster to train on than regular RF because finding thresholds is time-consuming for the algorithm on its own.

Can use ExtraTreeClassifier class, identical in structure to RF. ExtraTreeRegressor also exists.

Generally you can't tell if RF or Extra will be better unless you compare them directly.

### Feature Importance

Forests make it easy to meausre feature imporance. SKlearn measures importance by looking at how many nodes use the feature to reduce impurity on avg. This is a weighted average.

SCore is computed automatically and the results are scaled to equal 1. Results accessed with feature_importances_ variable. 

EG the following is used for Iris dataset. It shows petal length and width are most important.

In [71]:
from sklearn.datasets import load_iris
iris=load_iris()
rnd_clf = RandomForestClassifier(n_estimators=500,n_jobs=-1)
rnd_clf.fit(iris["data"],iris["target"])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=-1, oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

In [73]:
for name, score in zip(iris["feature_names"],rnd_clf.feature_importances_):
    print(name,score)

sepal length (cm) 0.10854327575984975
sepal width (cm) 0.02500943683325653
petal length (cm) 0.4318599718760198
petal width (cm) 0.43458731553087404


similarly on MNIST:

<img src="CH 07 Images/06.PNG" />

## Boosting

Boosting (hypothesis boosting) refers to any ensemble method that can combine several weak learners into a strong one. The idea is to train predictors sequentially, each trying to correct its predecessor. There are many methods, and ADA boost and Gradient boosting are most popular.

### AdaBoost

One way for a new predictor to correct its predeccesor is to pay more attention to training isntances that the predecessor underfitted. This results in new predictors focusing more on the hard cases. This is Adaboost.

EG when training an ADA classifier, algorithm first trains a base classifier (like a tree) and uses it to make predictions on the train set. The algorithm increases relative weight of misclassed instances, trains a second classifier with updated weights and again makes predictions, and this cycle continues:

<img src="CH 07 Images/07.PNG" />

Below we see decision boundaries of five consecutive predictors in the moon set. First classifier gets many wrong, their weights are updated. The right plot shows the same idea, but learning rate is halved (misclassed instance weights boosted half as much each iteration). This  looks similar to gradient descent, but instead of tweaking one predictor's parameters Adaboost adds predictors to the ensemble.

<img src="CH 07 Images/08.PNG" />

Once the predictors are trained, the ensemble makes predictions similar to bagging and pasting but the predictors have different weights depending on their overall accuracy on the weighted training set.

**Warning: because each predictor can only be trained after the previous predictor has been evaluated, it can't be parallelized. Thus it doesn't scale as well as bagging**

Let's look at the algorithm. Each weight, w, is set to 1/m. First predictor trained, weighted error r os computed.

<img src="CH 07 Images/09.PNG" />

The predictor's weight alpha is then computed, where n is the learning rate hyperparam. The moer accurate the predictor is the higher its weight will be. Random guessing will lead to a weight of ~0, and wrong guesses lead to negative weights.

<img src="CH 07 Images/10.PNG" />

NExt the algorithm updates its weights amd boosts weights of misclassed instances. Weights are all then normalized.

Finally, a new predictor is trained with the new weights and the process is repeated. It stops when the desired number of predictors is reached or when a perfect predictor is found.

To make predictions, AdaBoost computes the predictions of all the predictors and weighs them with their prediction weights. The predicted class is the one that receives the majority of weighted votes:

<img src="CH 07 Images/11.PNG" />

SKlearn uses a multiclass version of ADA called SAMME (stagewise additive modeling using a multiclass exponential loss function). When there are two classes SAMME is equivalent to ADA. If predictors use class probabilities rather than predictions, SAMME.R is used.

The following trains an ADA classifier based on 200 Descision stumps. A decision stump is a tree with depth=1, which is a single node +2 leafs. This is the default base estimator for ADA:

In [75]:
from sklearn.ensemble import AdaBoostClassifier

ada_clf=AdaBoostClassifier(DecisionTreeClassifier(max_depth=1),n_estimators=200,algorithm="SAMME.R",learning_rate=0.5)
ada_clf.fit(X_train,y_train)

AdaBoostClassifier(algorithm='SAMME.R',
                   base_estimator=DecisionTreeClassifier(class_weight=None,
                                                         criterion='gini',
                                                         max_depth=1,
                                                         max_features=None,
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=1,
                                                         min_samples_split=2,
                                                         min_weight_fraction_leaf=0.0,
                                                         presort=False,
                                                         random_state=None,
                             

**Tip: if the ADA ensemble is overfitting, reduce the # of estimators or more strongly regularize the base estimator.**

### Gradient Boosting

Another popular algorithm is Gradient Boosting. Also sequentially adds predictors, but instead of tweaking instance weights at each iteration it tries to fit new predictors to the *residual errors* made by the previous predictor.

Example with simple regression using trees as the base predictors. This is called Gradient Tree Boosting, or Gradient Boosted Regression Trees. First let's fit:

In [77]:
from sklearn.tree import DecisionTreeRegressor
tree_reg1=DecisionTreeRegressor(max_depth=2)
tree_reg1.fit(X,y)

DecisionTreeRegressor(criterion='mse', max_depth=2, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=None, splitter='best')

In [79]:
y2=y-tree_reg1.predict(X)
#Now we make a second tree based on residual errors...
#...made by the first tree
tree_reg2=DecisionTreeRegressor(max_depth=2)
tree_reg2.fit(X,y2)

DecisionTreeRegressor(criterion='mse', max_depth=2, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=None, splitter='best')

In [82]:
y3=y2-tree_reg2.predict(X)
#Now we make a third tree based on residual errors...
#...made by the second tree
tree_reg3=DecisionTreeRegressor(max_depth=2)
tree_reg3.fit(X,y3)

DecisionTreeRegressor(criterion='mse', max_depth=2, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=None, splitter='best')

Now we have an ensemble with three trees. We make predictions by adding the predictions of these trees

In [86]:
#y_pred = sum(tree.predict(X_new) for tree in (tree_reg1,tree_reg2, tree_reg3))

The firugre below shows predictions of the trees on the left and the ensemble's predictions on the right. The first row has one tree so the tree and ensemble match. The second row has a new tree trained on residuals and we can see on the left the predictions are equal to the sum of the first two trees. Sim for the third.

<img src="CH 07 Images/12.PNG" />

We see that the predictions get better as more trees are added. 

A simpler way to do this is with GradientBoostingRegressor. Sim to RF, it has hyperparams to control growth, ensemble training.

In [88]:
from sklearn.ensemble import GradientBoostingRegressor

gbrt = GradientBoostingRegressor(max_depth=2,n_estimators=3,learning_rate=1.0)
gbrt.fit(X,y)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=1.0, loss='ls', max_depth=2,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=3,
                          n_iter_no_change=None, presort='auto',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

learning_rate hyperparam scales the contribution of each tree. If this is too low, like 0.1 then you will need more trees but the prediction will generalize better. This is a technique called shrinkage.

Below we see two GBRT ensembles trained with low learning rate: one with too few trees and one with too many:

<img src="CH 07 Images/13.PNG" />

To find the optimal # of trees we can use Early Stopping with staged_predict() method: it returns an iterator over predictions made by the ensemble at each stage. The following trains a system with 120 trees, measures validation at each stage to find the optimal #. and then trains another GBRT using this #:

In [94]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X_train, X_val, y_train, y_val = train_test_split( X, y)

gbrt = GradientBoostingRegressor( max_depth = 2, n_estimators = 120) 

gbrt.fit( X_train, y_train)

errors = [mean_squared_error( y_val, y_pred) for y_pred in gbrt.staged_predict( X_val)] 

bst_n_estimators = np.argmin( errors) + 1

gbrt_best = GradientBoostingRegressor( max_depth = 2, n_estimators = bst_n_estimators) 

gbrt_best.fit( X_train, y_train)



GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=2,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=119,
                          n_iter_no_change=None, presort='auto',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

Validation errors are graphed on below, and the models predictions on the right:

<img src="CH 07 Images/14.PNG" />

We can also apply early stopping by stopping training early, using warm_start=True, which makes sklearn keep existing trees when fit() is called, allowing for incremental training. Below we see code that stops training when validation error doesn't improve for 5 iterations in a row:

In [96]:
#uses test data for the predictions
#substitute this for the validation set you actually want to use
gbrt = GradientBoostingRegressor(max_depth=2,warm_start=True)

min_val_error = float("inf")
error_going_up=0
for n_estimators in range(1,120):
    gbrt.n_estimators = n_estimators
    gbrt.fit(X_train, y_train)
    y_pred = gbrt.predict(X_test)
    val_error = mean_squared_error(y_test,y_pred)
    if val_error < min_val_error:
        min_val_error=val_error
        error_going_up=0
    else:
        error_going_up +=1
        if error_going_up ==5:
            break

GBRegressor class also supports subsample, whic specifies a fraction of the training instances to be used for each tree. EG is it =0.25, each tree will use a random 25% of the data. This increase bias and decreases variance. It also speeds up thge process. This is called Stochastic GBoosting.

Note you can use GBoosting with other cost functions as well.

An optimized version of Gradient boosting is available with the XGBoost library [extreme gradient boosting]. This aims to be faster, scalable, and portable. It is often part of winning entries in competitions. it's APi is similar to SKlearn's:

#need to install
<img src="CH 07 Images/15.PNG" />

### Stacking

The last method we will discuss is stacking, or stacked generalization. Based on a simple idea: instead of using trivial functions (like hard voting) to aggregate predictions, why don't we train a model to perform this aggregation on new instances? This concept is show  below. Each predictor makes its prediction, then the final predictor (blender or meta learner) takes these inputs and makes the final prediction.

<img src="CH 07 Images/16.PNG" />

To train the blender, commonly use a hold-out set.

First, the training set is splot into two sets. First set is used to train the predictors in the first layer.

<img src="CH 07 Images/17.PNG" />

The first layer's predictors are used to make predictions on the second set. This ensures that predictions are clean. For each instance in the holdout set there are 3 predicted values. These become inputs for a new training set, along with the true target value. The blender is trained on this set and lerns to predict the target value.

<img src="CH 07 Images/18.PNG" />

It is possible to train numerous blenders this way (one with linear reg, another with RF), to get a new layer of blenders. The trick would be to split training data into three subsets. First used to train the first layer, second for the second layer [the blenders], and the third for the final blender from the second layer's predictions.  Once this is done we can make a prediction for a new instance by going through each layer sequentially:

<img src="CH 07 Images/19.PNG" />

SKlearn doesn't support this directly, but it is possible to roll out our own implementation. OR we can use an open source library like DESlib.

